In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean, median
from scipy import stats
import math

sns.set_style('darkgrid')

from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.linear_model import LogisticRegression

## Análisis Exploratorio

In [ ]:
df = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
df.head()

In [ ]:
df.isnull().any()

In [ ]:
df.info()

### Análisis univariante

In [ ]:
fig = plt.figure(figsize=(20,12))

cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

count = 1

for col in cols:
    fig.add_subplot(3,4,count)
    sns.distplot(df[col])

    count += 1

plt.show()

In [ ]:
fig = plt.figure(figsize=(20,12))

cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

count = 1

for col in cols:
    fig.add_subplot(3,4,count)
    sns.boxplot(df[col])

    count += 1

plt.show()

Vamos a observar más de cerca algunos valores extremos:

In [ ]:
df.loc[df['citric acid'] == df['citric acid'].max()]

In [ ]:
df.loc[df['residual sugar'] == df['residual sugar'].max()]

In [ ]:
df.nlargest(2, 'total sulfur dioxide')

In [ ]:
df.nlargest(2, 'sulphates')

In [ ]:
df.loc[df['pH'] == df['pH'].max()]

In [ ]:
df.loc[df['alcohol'] == df['alcohol'].max()]

In [ ]:
df.loc[df['chlorides'] == df['chlorides'].max()]

Por lo general, los vinos con valores extremos (outliers) tienden a una calidad mediocre.

Vamos a ver cómo se distribuye la calidad:

In [ ]:
sns.distplot(df['quality'], kde=False)
plt.show()

In [ ]:
df.quality.unique()

Otra forma de ver cómo se distribuye la calidad es con la función de distribución acumulada (CDF). Observamos que los vinos con una calidad de 6 o menos suponen prácticamente el 90 % de nuestra muestra. Solo en el 10 % restante aproximadamente, se encuentran los vinos de mayor calidad.

In [ ]:
x = np.sort(df['quality'])
y = np.arange(1, len(x)+1) / len(x)

plt.plot(x, y, marker='.', linestyle='none')
plt.xlabel('Wine quality')
plt.ylabel('CDF')
plt.margins(0.02)

plt.show()


Veamos la estadística descriptiva de nuestras variables:

In [ ]:
df.describe(include='all')

El dióxido de azufre es un conservante del vino que se añade desde hace siglos. Dado que puede tener efectos adversos sobre la salud y sobre las propiedades del propio vino, existe una regulación acerca del mismo. En Europa las cantidades están más ajustadas, mientras que en EE.UU. podemos encontrar concentraciones por encima de 300.

Llaman la atención los valores extremos para esta variable, pero tienen esta explicación y no parecen ser errores de registro.

### Análisis multivariante - relaciones entre variables

In [ ]:
g = sns.PairGrid(df)
g.map(plt.scatter)
plt.show()

Con el pairgrid tenemos una visión amplia de las relaciones entre todas las variables, pero nos va a ser más útil estudiar las correlaciones en esta ocasión.

In [ ]:
pearson_corr = df.corr(method='pearson')
pearson_corr

In [ ]:
spearman_corr = df.corr(method='spearman')
spearman_corr

Las relaciones entre las variables son mayormente lineales y proceden de distribuciones normales (excepto el ácido cítrico), pero ante la presencia de valores atípicos (outliers) y cierta asimetría, es preferible quedarnos con el coeficiente de correlación de Spearman en este caso.

Usaremos el siguiente mapa de calor para tener una representación más clara:

In [ ]:
plt.figure(figsize=(16,12))
sns.heatmap(spearman_corr, xticklabels=spearman_corr.columns, yticklabels=spearman_corr.columns, center=0, annot=True)

plt.show()

Vamos a categorizar la calidad en distintos niveles y a estudiar un poco más de cerca algunas de las variables con las que se correlaciona más.

In [ ]:
bins = [0, 4, 7, 10]
labels = ['bad', 'normal', 'good']
df['quality'] = pd.cut(df['quality'], bins=bins, labels=labels)
df.head()

In [ ]:
sns.countplot(df['quality'])
plt.show()

In [ ]:
sns.violinplot(x=df['quality'], y=df['volatile acidity'])
plt.show()

La acidez volátil se pretende que sea lo más baja posible porque afecta al sabor. Es un conservante del vino. Vemos que los vinos de alta calidad tienden a tener una acidez volátil más baja.

In [ ]:
sns.violinplot(x=df['quality'], y=df['sulphates'])
plt.show()

In [ ]:
sns.violinplot(x=df['quality'], y=df['pH'])
plt.show()

Vemos cómo los vinos de alta calidad tienden ligeramente a tener un pH más ácido (en torno a 3,2) sin llegar a ser extremadamente ácidos (2,6) o muy planos (4).

In [ ]:
sns.violinplot(x=df['quality'], y=df['alcohol'])
plt.show()

Según los expertos, el grado de alcohol no es un condicionante de la calidad. Sin embargo, en esta ocasión vemos que los vinos de alta calidad tienden a tener un grado de alcohol mayor, pero están distribuidos en un rango aproximado de entre 8,5º y 15,5º.

In [ ]:
sns.violinplot(x=df['quality'], y=df['density'])
plt.show()

La densidad es una variable importante en la calidad del vino. Los de alta calidad están más distribuidos entre el rango de valores de densidad en mayor medida que los vinos de calidad pobre o normal.

### Contraste de hipótesis

En este caso vamos a testear si la diferencia de medias de grados de alcohol entre el grupo de vinos de mala calidad y de buena calidad es significativa.

H0: media alcohol vinos malos = media alcohol vinos buenos,
H1: media alcohol vinos malos NO es igual que la media alcohol vinos buenos

In [ ]:
bad_wines = df[df['quality'] == 'bad']
good_wines = df[df['quality'] == 'good']

In [ ]:
print('La media de alcohol de vinos de baja calidad es de:', bad_wines['alcohol'].mean())
print('El tamaño de la muestra es de:', len(bad_wines))
print('La varianza de la muestra es:', bad_wines['alcohol'].var())

print('La media de alcohol de vinos de alta calidad es de:', good_wines['alcohol'].mean())
print('El tamaño de la muestra es de:', len(good_wines))
print('La varianza de la muestra es:', good_wines['alcohol'].var())

In [ ]:
fig = plt.figure(figsize=(10,6))

fig.add_subplot(121)
sns.distplot(bad_wines['alcohol'])

fig.add_subplot(122)
sns.distplot(good_wines['alcohol'])

plt.show()

In [ ]:
stats.ttest_ind(bad_wines['alcohol'], good_wines['alcohol'], equal_var=False)

In [ ]:
se = math.sqrt((bad_wines['alcohol'].var() / len(bad_wines)) + (good_wines['alcohol'].var() / len(good_wines)))
print('El error estándar de la diferencia de medias entre los dos grupos es de:', se)

Rechazamos la hipótesis nula. Podemos decir que la diferencia entre las medias de grados de alcohol de los grupos de vinos de mala y buena calidad es estadísticamente significativa y es improbable que se deba al azar.

## Modelo de predicción de la calidad del vino

### Árbol de Decisión

In [ ]:
y = df['quality']

X = df.drop('quality', axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.33)

In [ ]:
dtc = DecisionTreeClassifier(max_leaf_nodes=20, random_state=1)

In [ ]:
dtc.fit(X_train, y_train)

In [ ]:
dt_pred = dtc.predict(X_valid)

In [ ]:
print('Algunas predicciones son:', list(dt_pred[:5]))
print('Comparadas con el objetivo:', list(y_valid[:5]))

In [ ]:
confusion_matrix(y_valid, dt_pred)

In [ ]:
accuracy_score(y_valid, dt_pred)

In [ ]:
dt_report = classification_report(y_valid, dt_pred, output_dict=True)

df_dt_report = pd.DataFrame(dt_report).transpose()
df_dt_report

Vemos que el "accuracy" es de más del 90 %. Buen resultado a primera vista. Sin embargo, viendo el resumen y la matriz de confusión, nos daremos cuenta de que el modelo no es capaz de reconocer los vinos clasificados como "buenos" o "malos" casi nunca.

### Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf_pred = rf.predict(X_valid)

In [ ]:
confusion_matrix(y_valid, rf_pred)

In [ ]:
accuracy_score(y_valid, rf_pred)

In [ ]:
rf_report = classification_report(y_valid, rf_pred, output_dict=True)

df_rf_report = pd.DataFrame(rf_report).transpose()
df_rf_report

En principio vemos que el Random Forest se desempeñará mejor en la tarea que estamos intentando resolver: ser capaces de predecir la calidad del vino. Para intentar mejorar este modelo, llevaremos a cabo una serie de transformaciones mientras jugamos con parámetros e ingeniería de variables.

Para empezar, dado que existen numerosos valores extremos en nuestro conjunto de datos, usaremos un normalizador robusto para ver qué ocurre con el modelo.

In [ ]:
scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

In [ ]:
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_valid)

In [ ]:
confusion_matrix(y_valid, rf_pred)

In [ ]:
rf_report = classification_report(y_valid, rf_pred, output_dict=True)

df_rf_report = pd.DataFrame(rf_report).transpose()
df_rf_report

No vemos mejoría o, al menos, no es significativa.

Importante: hay que tener en cuenta que hay una categoría muchísimo más grande que las otras dos. Hay muchos más registros con categoría "normal" que con "bueno" o "malo".

Es un caso de problema de clasificación no balanceado (imbalanced). En estos casos, fijarnos solamente en el "accuracy" no es una buena idea, no es una métrica buena en esta ocasión para medir el rendimiento de nuestro modelo.

A continuación, voy a crear más categorías dentro de calidad, para intentar que el conjunto de datos quede más equilibrado y los modelos puedan predecir mejor una u otra categoría. Otra opción sería realizar un sobremuestreo (oversampling).

#### Recategorizando

In [ ]:
df2 = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
bins = [0, 4, 5, 6, 10]
labels = ['bad', 'normal', 'good', 'very good']
df2['quality'] = pd.cut(df2['quality'], bins=bins, labels=labels)
df2.head()

In [ ]:
sns.countplot(df2['quality'])
plt.show()

Vemos que sigue habiendo, como ya esperábamos, grandes diferencias entre categorías.

Crearemos el modelo de predicción de nuevo y, si no conseguimos buena precisión, podemos pasar a hacer ingeniería de variables o sobremuestreo.

In [ ]:
y = df2['quality']
X = df2.drop('quality', axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.33)

In [ ]:
rf2 = RandomForestClassifier()

rf2.fit(X_train, y_train)

rf2_pred = rf2.predict(X_valid)

In [ ]:
confusion_matrix(y_valid, rf2_pred)

In [ ]:
rf2_report = classification_report(y_valid, rf2_pred, output_dict=True)

df_rf2_report = pd.DataFrame(rf2_report).transpose()
df_rf2_report

Después de crear nuevas categorías, vemos que las métricas de precisión mejoran y el F1 también. Sin embargo, el modelo sigue sin ver el patrón para vinos de categoría mala. Al aumentar el tamaño del conjunto de datos de validación, los resultados mejoraron también. Entiendo que el problema en este caso es por el tamaño de las muestras dentro de cada categoría.

#### Ingeniería de variables (feature engineering)

Las variables que más se correlacionan con la calidad son la acidez volátil, los sulfatos y el alcohol.

Vamos a generar una nueva variable a partir del alcohol, por ejemplo, para ayudar al modelo a discriminar entre categorías.

In [ ]:
df2['alcohol squared'] = df2['alcohol'] ** 2

In [ ]:
y = df2['quality']
X = df2.drop('quality', axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.33)

In [ ]:
rf2.fit(X_train, y_train)

rf2_pred = rf2.predict(X_valid)

In [ ]:
confusion_matrix(y_valid, rf2_pred)

In [ ]:
rf2_report = classification_report(y_valid, rf2_pred, output_dict=True)

df_rf2_report = pd.DataFrame(rf2_report).transpose()
df_rf2_report

#### Re-sampling

In [ ]:
df3 = df2.copy()

In [ ]:
df3.columns

In [ ]:
y = df3['quality']
X = df3.drop('quality', axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.33)

Dado que el tamaño de la muestra que cae en cada categoría está desequilibrado, vamos a usar técnicas de sobremuestreo (oversampling).

Empezamos realizando la técnica de oversampling más sencilla con el siguiente algoritmo:

In [ ]:
ros = RandomOverSampler(random_state=0)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
sns.countplot(y_train_resampled)
plt.show()

In [ ]:
rf2.fit(X_train_resampled, y_train_resampled)

rf2_pred = rf2.predict(X_valid)

In [ ]:
confusion_matrix(y_valid, rf2_pred)

In [ ]:
rf2_report = classification_report(y_valid, rf2_pred, output_dict=True)

df_rf2_report = pd.DataFrame(rf2_report).transpose()
df_rf2_report

Usando otro algoritmo de oversampling llamado SMOTE:

In [ ]:
sm = SMOTE(random_state=42, k_neighbors=8)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [ ]:
y_train_res.value_counts()

In [ ]:
rf2.fit(X_train_res, y_train_res)

rf2_pred = rf2.predict(X_valid)

In [ ]:
confusion_matrix(y_valid, rf2_pred)

In [ ]:
rf2_report = classification_report(y_valid, rf2_pred, output_dict=True)

df_rf2_report = pd.DataFrame(rf2_report).transpose()
df_rf2_report

No somos capaces de ver una mejora sustancial. Pasamos a hacer ingeniería de varibales con el target, convirtiéndolo a variable binaria.

### Regresión logística

In [ ]:
data = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
data['quality'] = data.quality.apply(lambda x: 1 if x > 5 else 0)

In [ ]:
data['quality'].head()

In [ ]:
sns.countplot(data['quality'])
plt.show()

In [ ]:
y = data['quality']
X = data.drop('quality', axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
logit = LogisticRegression()

In [ ]:
logit.fit(X_train, y_train)

logit_pred = logit.predict(X_valid)

In [ ]:
confusion_matrix(y_valid, logit_pred)

In [ ]:
logit_report = classification_report(y_valid, logit_pred, output_dict=True)

df_logit_report = pd.DataFrame(logit_report).transpose()
df_logit_report

In [ ]:
roc_auc_score(y_valid, logit_pred)

#### Ingeniería de variables

Incluyo nuevamente la variable de grados de alcohol elevados al cuadrado, para que ayude al modelo a discriminar mejor entre las categorías.

In [ ]:
data['alcohol squared'] = data.alcohol ** 2

In [ ]:
y = data.quality
X = data.drop('quality', axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
logit2 = LogisticRegression()

In [ ]:
logit2.fit(X_train, y_train)

logit2_pred = logit2.predict(X_valid)

In [ ]:
confusion_matrix(y_valid, logit2_pred)

In [ ]:
logit2_report = classification_report(y_valid, logit2_pred, output_dict=True)

df_logit2_report = pd.DataFrame(logit2_report).transpose()
df_logit2_report

In [ ]:
roc_auc_score(y_valid, logit2_pred)

## Conclusiones

La intención de este proyecto era profundizar un poco más en nuevos conceptos. A medida que el proyecto ha ido avanzando, he podido hacer paréntesis para detenerme y comprender nuevos términos, algoritmos, técnicas, etc.

Hemos podido ver cómo las métricas de los modelos han ido evolucionando a medida que añadíamos, quitábamos o modificábamos algo. Eso es importante para mejorar la comprensión de lo que estamos haciendo. El "accuracy", en un conjunto de datos no balanceado como el que teníamos, no era una métrica adecuada. El objetivo ha sido ir mejorando la precisión (y el F1 score en conjunto) hasta alcanzar niveles aceptables. 